In [2]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(path_to_dataset) #path of the dataset
df.columns= ["clauses", "critical"]


In [3]:
df['critical'] = df['critical'].apply(lambda x: 1 if x == 'critical' else 0)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['clauses'], df['critical'])


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(Path_bert_en_uncased_preprocess_3)#Path of the uncased preprocessor of BERT
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    Path_small_bert_bert_en_uncased_L-4_H-256_A-4_2,
    trainable=True) #Path of small_bert_bert_en_uncased_L-4_H-256_A-4_2
outputs = encoder(encoder_inputs)


In [7]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [8]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [9]:
METRICS = [ tf.keras.metrics.BinaryAccuracy(name='accuracy'), tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

In [10]:
history = model.fit(X_train, y_train, epochs=5)        

Epoch 1/5
29/29 [==============================] - 34s 985ms/step - loss: 0.7185 - accuracy: 0.6103 - precision: 0.5892 - recall: 0.6014
Epoch 2/5
29/29 [==============================] - 29s 1s/step - loss: 0.4478 - accuracy: 0.8408 - precision: 0.8449 - recall: 0.8157
Epoch 3/5
29/29 [==============================] - 56s 2s/step - loss: 0.2753 - accuracy: 0.9133 - precision: 0.9319 - recall: 0.8825
Epoch 4/5
29/29 [==============================] - 60s 2s/step - loss: 0.2552 - accuracy: 0.9133 - precision: 0.8953 - recall: 0.9263
Epoch 5/5
29/29 [==============================] - 60s 2s/step - loss: 0.2159 - accuracy: 0.9297 - precision: 0.9405 - recall: 0.9101


In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
for i in range(len(X_test)):
    print(i, " ", X_test.iloc[i])
    print(y_predicted)    


In [12]:
model.evaluate(X_test, y_test)

10/10 [==============================] - 4s 305ms/step - loss: 0.2613 - accuracy: 0.9211 - precision: 0.9524 - recall: 0.8917


[0.26129451394081116,
 0.9210526347160339,
 0.9523809552192688,
 0.8917197585105896]